In [1]:
import re
import math
import random
import joblib
import numpy as np
import statistics as stats
from collections import Counter
from tqdm.auto import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

/Users/huynhcongbang/ai_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cấu hình Regex và Stopwords
PUNCT_RE = re.compile(r"[^\w\s]", re.UNICODE)
STOPWORDS = set(["the", "be", "to", "of", "and", "a", "in", "that", "have", "i", 
                 "it", "for", "not", "on", "with", "he", "as", "you", "do", "at"])

def count_syllables(word):
    """Đếm âm tiết đơn giản để tính chỉ số Readability"""
    word = word.lower()
    if len(word) <= 3: return 1
    word = re.sub(r'(?:[^laeiouy]es|ed|[^laeiouy]e)$', '', word)
    word = re.sub(r'^y', '', word)
    syllables = len(re.findall(r'[aeiouy]{1,2}', word))
    return max(1, syllables)

def shannon_entropy(text):
    """Tính độ hỗn loạn thông tin (Entropy) của ký tự"""
    if not text: return 0
    counts = Counter(text)
    total = len(text)
    return -sum((cnt / total) * math.log2(cnt / total) for cnt in counts.values())

In [3]:
def extract_features_pro(text):
    words = text.split()
    sentences = [s for s in re.split(r"[.!?]", text) if s.strip()]
    if not sentences: sentences = [text]
    
    num_words = len(words)
    num_sentences = len(sentences) # Sửa lỗi chia cho 0
    avg_sent_len = num_words / max(1, num_sentences)
    
    # Burstiness (Độ biến thiên độ dài câu)
    sent_lens = [len(s.split()) for s in sentences]
    burstiness = stats.pstdev(sent_lens) / avg_sent_len if avg_sent_len > 0 else 0
    
    # Readability (Flesch Reading Ease giả lập)
    num_syllables = sum(count_syllables(w) for w in words)
    flesch_score = 206.835 - 1.015 * avg_sent_len - 84.6 * (num_syllables / max(1, num_words))
    
    # Stopword Ratio (AI thường dùng tỷ lệ từ dừng rất chuẩn mực)
    stop_count = sum(1 for w in words if w.lower() in STOPWORDS)
    
    return {
        "len_words": num_words,
        "ttr": len(set(words)) / max(1, num_words),
        "punct_ratio": len(PUNCT_RE.findall(text)) / max(1, len(text)),
        "avg_sent_len": avg_sent_len,
        "burstiness": burstiness,
        "entropy": shannon_entropy(text),        # Mới
        "flesch_score": flesch_score,            # Mới
        "stopword_ratio": stop_count / max(1, num_words) # Mới
    }

In [4]:
class PPLScorer:
    def __init__(self, model_name="distilgpt2"):
        print(f"Loading LM: {model_name}...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.model.eval()

    def perplexity(self, text):
        # Cắt ngắn text xuống 512 tokens để tránh lỗi tràn bộ nhớ
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            loss = self.model(**inputs, labels=inputs["input_ids"]).loss
        return float(torch.exp(loss))

In [5]:
def load_texts(human_file="human.txt", ai_file="ai.txt"):
    texts, labels = [], []
    for f_path, label in [(human_file, 0), (ai_file, 1)]:
        try:
            with open(f_path, encoding="utf-8") as f:
                content = f.read().split("\n\n")
                print(f"Loaded {len(content)} samples from {f_path}")
                for t in content:
                    if len(t.strip()) > 30:
                        texts.append(t.strip())
                        labels.append(label)
        except Exception as e:
            print(f"Error loading {f_path}: {e}")
    return texts, labels

# Thực thi load data
texts, labels = load_texts()

# Shuffle dữ liệu
combined = list(zip(texts, labels))
random.shuffle(combined)
texts, labels = zip(*combined)
texts, labels = list(texts), list(labels)
print(f"🔥 Tổng cộng: {len(texts)} mẫu dữ liệu sẵn sàng.")

Loaded 15395 samples from human.txt
Loaded 15527 samples from ai.txt
🔥 Tổng cộng: 30922 mẫu dữ liệu sẵn sàng.


In [6]:
scorer = PPLScorer()
X_data = []
feat_names = []

print("⏳ Đang trích xuất features (vui lòng đợi)...")
for t in tqdm(texts):
    f = extract_features_pro(t)
    f["ppl"] = scorer.perplexity(t)
    X_data.append(list(f.values()))
    if not feat_names: feat_names = list(f.keys())

X = np.array(X_data)
y = np.array(labels)
print("✅ Đã trích xuất xong đặc trưng!")

Loading LM: distilgpt2...
⏳ Đang trích xuất features (vui lòng đợi)...


100%|███████████████████████████████████| 30922/30922 [1:51:55<00:00,  4.60it/s]

✅ Đã trích xuất xong đặc trưng!


In [7]:
# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Định nghĩa 3 model con
clf1 = LogisticRegression(random_state=1, max_iter=1000)
clf2 = RandomForestClassifier(n_estimators=100, random_state=1)
clf3 = SVC(probability=True, random_state=1)

# Kết hợp bằng Soft Voting
voting_clf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('svm', clf3)],
    voting='soft'
)

# Pipeline với PowerTransformer (tốt hơn StandardScaler cho dữ liệu bị lệch)
pipe = Pipeline([
    ("scaler", PowerTransformer()), 
    ("voting", voting_clf)
])

print("🚀 Đang huấn luyện Ensemble Model...")
pipe.fit(X_train, y_train)
print("🏁 Huấn luyện hoàn tất!")

🚀 Đang huấn luyện Ensemble Model...
🏁 Huấn luyện hoàn tất!


In [8]:
print("\n" + "="*40)
print("🏆 KẾT QUẢ ĐÁNH GIÁ )
print("="*40)

y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred, digits=4))
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

# Cross Validation check
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy (5-fold): {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Lưu model
joblib.dump({"pipe": pipe, "feat_names": feat_names}, "aidetect_model_pro.joblib")
print(f"\n💾 Đã lưu model vào: aidetect_model_pro.joblib")


🏆 KẾT QUẢ ĐÁNH GIÁ (PRO VERSION)
              precision    recall  f1-score   support

           0     0.9609    0.9581    0.9595      3079
           1     0.9586    0.9614    0.9600      3106

    accuracy                         0.9597      6185
   macro avg     0.9597    0.9597    0.9597      6185
weighted avg     0.9597    0.9597    0.9597      6185

ROC-AUC Score: 0.9924
Cross-Validation Accuracy (5-fold): 0.9597 (+/- 0.0041)

💾 Đã lưu model vào: aidetect_model_pro.joblib
